In [2]:
import pandas as pd
from scipy.stats import chi2_contingency

In [185]:
self = pd.read_excel("./0-250_self分类.xlsx") # 读取self的分类
self_comment = pd.read_excel("./total_result.xlsx") # 读取每一个self它所相关的comments的数量

In [187]:
#寻找self中，相关话题的index/find index relevant to self-presentation
num_find_friends = 0
num_find_partner = 0
num_find_sex = 0
num_share = 0
num_unrelated = 0

index_find_friends = []
index_find_partner = []
index_find_sex = []
index_share = []
index_unrelated = []

for i in range(self.shape[0]):
    result = self['找朋友'][i]
    if result == 1:
        num_find_friends += 1
        index_find_friends.append(i)
        
for i in range(self.shape[0]):
    result = self['找对象'][i]
    if result == 1:
        num_find_partner += 1
        index_find_partner.append(i)
        
for i in range(self.shape[0]):
    result = self['找炮友'][i]
    if result == 1:
        num_find_sex += 1
        index_find_sex.append(i)
        
        
for i in range(self.shape[0]):
    result = self['分享'][i]
    if result == 1:
        num_share += 1
        index_share.append(i)
        
for i in range(self.shape[0]):
    result = self['和以上无关'][i]
    if result == 1:
        num_unrelated += 1
        index_unrelated.append(i)
        

In [188]:
self_motivation = {'找朋友': index_find_friends, '找对象': index_find_partner , 
                   '找炮友': index_find_sex, '分享': index_share , 
                   '和以上无关': index_unrelated}

In [189]:
# 开始处理文件格式/begin process format
comments_motivation = ["Wanna know each other", "For more information", "Expressing emotional opinions and sharing experiences",
                       "Interaction", "Unrelated"]
result = []
for topic in comments_motivation:
    temp = []
    for key in self_motivation.keys():
#         print(key)
        count = 0
        list_index = self_motivation[key]
        hang = []
        for index in list_index:
            a = self_comment.loc[self_comment["index"] == index].index.values
            for val in a:
                hang.append(int(val))    
        for num_hang in hang:
            add = self_comment.loc[num_hang,topic]
            count += add
        temp.append(count)
    result.append(temp)

In [190]:
# 加载模板/load
template = pd.read_excel("./correlation.xlsx")

In [192]:
template['Wanna know each other'] = result[0]
template['For more information'] = result[1]
template['Expressing emotional opinions and sharing experiences'] = result[2]
template['Interaction'] = result[3]
template['Unrelated'] = result[4]

In [193]:
template

,Wanna know each other,For more information,Expressing emotional opinions and sharing experiences,Interaction,Unrelated
0,522.0,300.0,1812.0,7449.0,1824.0
1,1625.0,719.0,4516.0,19886.0,4894.0
2,27.0,14.0,108.0,497.0,125.0
3,1632.0,831.0,5364.0,22311.0,5589.0
4,86.0,55.0,438.0,1813.0,428.0


In [194]:
#添加总的数量/add total number
total_self = []
for i in range(template.shape[0]):
    a = template.loc[i]
    count = 0
    for item in a:
        count += item
    total_self.append(count)

template['total'] = total_self

In [195]:
new_comments_motivation = ["Wanna know each other", "For more information", "Expressing emotional opinions and sharing experiences",
                       "Interaction", "Unrelated", "total"]
total_comment = []
for i in new_comments_motivation:
    count = 0
    for num in template[i]:
        count += num
    total_comment.append(count)
    
template.loc[5] = total_comment

In [196]:
template

,Wanna know each other,For more information,Expressing emotional opinions and sharing experiences,Interaction,Unrelated,total
0,522.0,300.0,1812.0,7449.0,1824.0,11907.0
1,1625.0,719.0,4516.0,19886.0,4894.0,31640.0
2,27.0,14.0,108.0,497.0,125.0,771.0
3,1632.0,831.0,5364.0,22311.0,5589.0,35727.0
4,86.0,55.0,438.0,1813.0,428.0,2820.0
5,3892.0,1919.0,12238.0,51956.0,12860.0,82865.0


In [197]:
# 修改名字/change name
df_final = template.rename(index = {0: '找朋友', 1: '找对象', 2: '找炮友', 3: '分享', 4: '和以上无关', 5: 'total'})

In [198]:
df_final

,Wanna know each other,For more information,Expressing emotional opinions and sharing experiences,Interaction,Unrelated,total
找朋友,522.0,300.0,1812.0,7449.0,1824.0,11907.0
找对象,1625.0,719.0,4516.0,19886.0,4894.0,31640.0
找炮友,27.0,14.0,108.0,497.0,125.0,771.0
分享,1632.0,831.0,5364.0,22311.0,5589.0,35727.0
和以上无关,86.0,55.0,438.0,1813.0,428.0,2820.0
total,3892.0,1919.0,12238.0,51956.0,12860.0,82865.0


In [181]:
data = df_final
stat, p, dof, expected = chi2_contingency(df_final)

# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.0008449550722031457
Dependent (reject H0)


这里说明了self的motivation和comments的motivation相关程度非常高
相应的，还可以做话题的相关程度分析
this example/test shows the first 250 comments is dependent to self-presentations 

In [12]:
def chi2_test(data):
    stat, p, dof, expected = chi2_contingency(data)
    # interpret p-value
    alpha = 0.05
    print("p value is " + str(p))
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (H0 holds true)')

In [3]:
data = pd.read_excel("/Users/rampageyao/Desktop/Research/Data/Chi_Test.xlsx")
data.head()

,Unnamed: 0,want relation,want more info,share opinion,relationship,social,sex,appearance,location,education,personal info,daily life
0,want bf,4314,4377,7178,8964,2747,3232,3753,5906,5290,4994,3366
1,want sex partner,175,164,317,403,88,167,165,245,212,217,142
2,want friend,1045,1270,2101,2570,713,870,1004,1528,1498,1290,975
3,share,5211,5471,9489,11690,3329,4104,4841,7217,6652,6331,4348
4,relationship,3881,4125,7190,8858,2586,3214,3616,5512,4879,4790,3207


In [7]:
motiv_motiv = data.iloc[:4,1:4]
motiv_motiv.head()

,want relation,want more info,share opinion
0,4314,4377,7178
1,175,164,317
2,1045,1270,2101
3,5211,5471,9489


In [13]:
chi2_test(motiv_motiv)

p value is 2.372473354775801e-05
Dependent (reject H0)


In [8]:
motiv_topic = data.iloc[:4,4:]
motiv_topic.head()

,relationship,social,sex,appearance,location,education,personal info,daily life
0,8964,2747,3232,3753,5906,5290,4994,3366
1,403,88,167,165,245,212,217,142
2,2570,713,870,1004,1528,1498,1290,975
3,11690,3329,4104,4841,7217,6652,6331,4348


In [14]:
chi2_test(motiv_topic)

p value is 0.008082235848397696
Dependent (reject H0)


In [9]:
topic_motiv = data.iloc[4:,1:4]
topic_motiv.head()

,want relation,want more info,share opinion
4,3881,4125,7190
5,1472,1607,2959
6,2936,2933,4651
7,2507,2528,4288
8,5130,5277,8760


In [15]:
chi2_test(topic_motiv)

p value is 6.678632133967893e-07
Dependent (reject H0)


In [10]:
topic_topic = data.iloc[4:,4:]
topic_topic.head()

,relationship,social,sex,appearance,location,education,personal info,daily life
4,8858,2586,3214,3616,5512,4879,4790,3207
5,3610,947,1155,1455,2129,1995,1832,1386
6,5804,1971,2089,2402,3861,3388,3204,2116
7,5417,1598,1895,2224,3315,2991,2974,1998
8,11083,3267,3859,4628,6929,6262,6054,4101


In [16]:
chi2_test(topic_topic)

p value is 5.047636959472858e-05
Dependent (reject H0)
